In [1]:
%run functions.py
%run meps_region_states.py

# Create MEPS_Region_States SQL Table

In [2]:
sql_create_table('MEPS_region_states', MEPS_region_states)

MEPS_region_states table created in DB
Could not delete dataframe from memory as MEPS_region_states Dataframe does not exist


In [3]:
ingred_prod_ndc = db_query(f"""SELECT * FROM joey_query""")
ingred_prod_ndc

,index,medication_ingredient_rxcui,medication_ingredient_name,medication_ingredient_tty,medication_product_rxcui,medication_product_name,medication_product_tty,dose_form_rxcui,dose_form_name,dose_form_tty,medication_ndc
0,0,5499,hydrogen peroxide,IN,91349,hydrogen peroxide 30 MG/ML Topical Solution,SCD,316986,Topical Solution,DF,00295117904
1,1,5499,hydrogen peroxide,IN,91349,hydrogen peroxide 30 MG/ML Topical Solution,SCD,316986,Topical Solution,DF,00295117916
2,2,5499,hydrogen peroxide,IN,91349,hydrogen peroxide 30 MG/ML Topical Solution,SCD,316986,Topical Solution,DF,00363026816
3,3,5499,hydrogen peroxide,IN,91349,hydrogen peroxide 30 MG/ML Topical Solution,SCD,316986,Topical Solution,DF,00363026832
4,4,5499,hydrogen peroxide,IN,91349,hydrogen peroxide 30 MG/ML Topical Solution,SCD,316986,Topical Solution,DF,00363087143
...,...,...,...,...,...,...,...,...,...,...,...
292790,292790,1547611,oritavancin,IN,2532366,oritavancin 1200 MG Injection [Kimyrsa],SBD,1649574,Injection,DF,70842022501
292791,292791,8163,phenylephrine,IN,2533434,5 ML phenylephrine hydrochloride 0.1 MG/ML Inj...,SBD,1649574,Injection,DF,71863020205
292792,292792,8163,phenylephrine,IN,2533434,5 ML phenylephrine hydrochloride 0.1 MG/ML Inj...,SBD,1649574,Injection,DF,71863020206
292793,292793,8163,phenylephrine,IN,2533437,1 ML phenylephrine hydrochloride 10 MG/ML Inje...,SBD,1649574,Injection,DF,71863020301


In [4]:
ingred_prod_ndc = 'joey_query'

In [5]:
drug_category_level = ['medication_ingredient_name', 'name_strength_df']

# Create Distribution DataFrames

In [6]:
#Using medication_ingredient, dosage form, and strength (pulled from the product_name) RxCUIs, create a distribution of medications across a therapeutic class that is segmented by age, gender, and state.
#SQLite does not have REGEX, so used multiple REPLACE functions to remove non-numerical characters.
#"Sex" assignments are from MEPS, source: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=PROJYR15&varName=SEX 
#Age buckets may be recategorized.

drug_distrib_dict = {}

for level in drug_category_level: 
    drug_distrib_dict[level] = db_query(f"""
    WITH meps_rxcui AS 
    (SELECT DISTINCT 
    t1.medication_ingredient_rxcui, 
    t1.medication_ingredient_name,
    REPLACE(REPLACE(
    REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE
    (REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(
    REPLACE(t1.medication_product_name, 'a', '')
     , 'b', ''), 'c', ''), 'd', ''), 'e', ''), 'f', ''), 'g', ''), 'h', ''), 'i', ''), 'j', ''), 'k', '')
     , 'l', ''), 'm', ''), 'n', ''), 'o', ''), 'p', ''), 'q', ''), 'r', ''), 's', ''), 't', ''), 'u', '')
     , 'v', ''), 'w', ''), 'x', ''), 'y', ''), 'z', ''), '[', ''), ']', '')
    AS medication_strength,
    t1.dose_form_rxcui,
    t1.dose_form_name,
    t2.dupersid,
    t2.perwt18f,
    CASE WHEN t3.sex = 1 THEN 'Male' 
    WHEN t3.sex = 2 THEN 'Female'
    END AS gender, 
    CASE WHEN t3.age18x < 4 THEN 'Under 4'
    WHEN t3.age18x BETWEEN 4 AND 7 THEN '4 - 7'
    WHEN t3.age18x BETWEEN 8 AND 11 THEN '8 - 11'
    WHEN t3.age18x BETWEEN 12 AND 18 THEN '12 - 18'
    WHEN t3.age18x BETWEEN 19 AND 49 THEN '19 - 49'
    WHEN t3.age18x BETWEEN 50 AND 64 THEN '50 - 64'
    WHEN t3.age18x >= 65 THEN '65 and up'
    END AS age,
    t4.states AS state
    FROM {ingred_prod_ndc} AS t1
    INNER JOIN meps_prescribedmeds AS t2
    ON t1.medication_ndc = t2.rxndc
    INNER JOIN meps_patientdemographics AS t3
    ON t2.dupersid = t3.dupersid
    INNER JOIN meps_region_states AS t4
    ON t3.region18 = t4.region_value
    WHERE t1.medication_ingredient_name LIKE '%levothyroxine%'
    OR t1.medication_ingredient_name LIKE '%liothyronine%'
    OR t1.medication_ingredient_name LIKE '%thyroid%'
    OR t2.TC1S1 = 103 -- thyroid class
    OR t2.TC1S2 = 103 
    OR t2.TC1S3 = 103 
    OR t2.TC1 = 103 -- thyroid class
    OR t2.TC2 = 103 
    OR t2.TC3 = 103
    )

    , patient_count AS
    (SELECT DISTINCT
    medication_ingredient_rxcui,
    medication_ingredient_name,
    REPLACE(
    REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(
    REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(medication_strength, 
    'A', '') , 'B', ''), 'C', ''), 'D', ''), 'E', ''), 'F', ''), 'G', ''), 'H', ''), 'I', ''), 'J', ''), 
     'K', ''), 'L', ''), 'M', ''), 'N', ''), 'O', ''), 'P', ''), 'Q', ''), 'R', ''), 'S', ''), 'T', ''), 
     'U', ''), 'V', ''), 'W', ''), 'X', ''), 'Y', ''), 'Z', '')
     , '.', '_') AS medication_strength, --couldn't fit all in 1 subquery ; also convert decimals to underscores
    dose_form_rxcui,
    dose_form_name,
    gender,
    age,
    state,
    perwt18f,
    COUNT(DISTINCT dupersid) AS patient_count
    FROM meps_rxcui
    GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9
    )

    , weighted_patient_count AS
    (SELECT DISTINCT
    medication_ingredient_name,
    medication_ingredient_rxcui,
    REPLACE(REPLACE(REPLACE(TRIM(medication_ingredient_name, ' ') || TRIM(medication_strength, ' ') || TRIM(dose_form_name, ' '), '(', ''), ')', ''), ' ', '_') AS name_strength_df,
    dose_form_rxcui,
    gender,
    age,
    state,
    SUM(patient_count*perwt18f) AS weighted_patient_count
    FROM patient_count
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    )

    , patients_by_demographics_prescribed_med AS 
    (SELECT DISTINCT t1.*, t2.total_ingred_prescribe_patients,
    t1.demog_ingred_prescribe_patients/t2.total_ingred_prescribe_patients*100 AS percent_ingred_prescribe_patients
    FROM (
    SELECT DISTINCT 'thyroid' AS disease_class, age, gender, state, 
    'prescribe_' || {level} AS ingredient_prescribe_category,
    SUM(weighted_patient_count) AS demog_ingred_prescribe_patients
    FROM weighted_patient_count
    GROUP BY 1, 2, 3, 4, 5) AS t1
    INNER JOIN (SELECT DISTINCT
    'thyroid' AS disease_class, age, gender, state,
    SUM(weighted_patient_count) AS total_ingred_prescribe_patients
    FROM weighted_patient_count
    GROUP BY 1, 2, 3, 4) AS t2
    ON t1.disease_class = t2.disease_class
    AND t1.age = t2.age
    AND t1.gender = t2.gender
    AND t1.state = t2.state
    )
    
    SELECT DISTINCT *
    FROM patients_by_demographics_prescribed_med
    """)
    sql_create_table(level+'_distrib',drug_distrib_dict[level])
    

medication_ingredient_name_distrib table created in DB
Could not delete dataframe from memory as medication_ingredient_name_distrib Dataframe does not exist
name_strength_df_distrib table created in DB
Could not delete dataframe from memory as name_strength_df_distrib Dataframe does not exist


# Create Drug-level Distribution CSV file

In [7]:
ingred_prescribe_categories = db_query(f"""
    SELECT DISTINCT ingredient_prescribe_category, LENGTH(ingredient_prescribe_category)
    FROM name_strength_df_distrib AS t1
    ORDER BY 1
    """)

ingred_prescribe_category_list = ingred_prescribe_categories['ingredient_prescribe_category'].unique().tolist()
ingred_prescribe_category_list

['prescribe_levothyroxine0_013Oral_Capsule',
 'prescribe_levothyroxine0_025Oral_Tablet',
 'prescribe_levothyroxine0_05Oral_Capsule',
 'prescribe_levothyroxine0_05Oral_Tablet',
 'prescribe_levothyroxine0_075Oral_Tablet',
 'prescribe_levothyroxine0_088Oral_Capsule',
 'prescribe_levothyroxine0_088Oral_Tablet',
 'prescribe_levothyroxine0_112Oral_Tablet',
 'prescribe_levothyroxine0_125Oral_Tablet',
 'prescribe_levothyroxine0_137Oral_Tablet',
 'prescribe_levothyroxine0_15Oral_Tablet',
 'prescribe_levothyroxine0_175Oral_Tablet',
 'prescribe_levothyroxine0_1Oral_Capsule',
 'prescribe_levothyroxine0_1Oral_Tablet',
 'prescribe_levothyroxine0_2Oral_Tablet',
 'prescribe_levothyroxine0_3Oral_Tablet',
 'prescribe_liothyronine0_005Oral_Tablet',
 'prescribe_liothyronine0_025Oral_Tablet',
 'prescribe_thyroid_USP_120Oral_Tablet',
 'prescribe_thyroid_USP_15Oral_Tablet',
 'prescribe_thyroid_USP_180Oral_Tablet',
 'prescribe_thyroid_USP_30Oral_Tablet',
 'prescribe_thyroid_USP_60Oral_Tablet',
 'prescribe_thy

In [8]:
drug_distrib = db_query(f"""
    SELECT DISTINCT t1.age, t1.gender, t1.state, t2.prescribe_levothyroxine, t3.prescribe_liothyronine, t4.prescribe_thyroid_usp
    FROM medication_ingredient_name_distrib AS t1
    LEFT JOIN 
    (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS prescribe_levothyroxine
    FROM medication_ingredient_name_distrib
    WHERE ingredient_prescribe_category = 'prescribe_levothyroxine') AS t2
    ON t1.age = t2.age
    AND t1.gender = t2.gender
    AND t1.state = t2.state 
    LEFT JOIN 
    (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS prescribe_liothyronine
    FROM medication_ingredient_name_distrib
    WHERE ingredient_prescribe_category LIKE '%prescribe_liothyronine%') AS t3
    ON t1.age = t3.age
    AND t1.gender = t3.gender
    AND t1.state = t3.state 
    LEFT JOIN 
    (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS prescribe_thyroid_usp
    FROM medication_ingredient_name_distrib
    WHERE ingredient_prescribe_category LIKE '%prescribe_thyroid (USP)%') AS t4
    ON t1.age = t4.age
    AND t1.gender = t4.gender
    AND t1.state = t4.state 
    """)

drug_distrib.to_csv('thyroid_drug_distrib.csv', index = False)
drug_distrib

,age,gender,state,prescribe_levothyroxine,prescribe_liothyronine,prescribe_thyroid_usp
0,12 - 18,Female,Alabama,100.0,NaN,NaN
1,12 - 18,Female,Alaska,100.0,NaN,NaN
2,12 - 18,Female,Arizona,100.0,NaN,NaN
3,12 - 18,Female,Arkansas,100.0,NaN,NaN
4,12 - 18,Female,California,100.0,NaN,NaN
...,...,...,...,...,...,...
469,8 - 11,Male,Utah,100.0,NaN,NaN
470,8 - 11,Male,Virginia,100.0,NaN,NaN
471,8 - 11,Male,Washington,100.0,NaN,NaN
472,8 - 11,Male,West Virginia,100.0,NaN,NaN


# Create Drug-Strength-DF-level Distribution CSV File

In [9]:
ingred_prescribe_categories = db_query(f"""
    SELECT DISTINCT ingredient_prescribe_category, LENGTH(ingredient_prescribe_category)
    FROM name_strength_df_distrib AS t1
    ORDER BY 1
    """)

ingred_prescribe_category_list = ingred_prescribe_categories['ingredient_prescribe_category'].unique().tolist()
ingred_prescribe_category_list

['prescribe_levothyroxine0_013Oral_Capsule',
 'prescribe_levothyroxine0_025Oral_Tablet',
 'prescribe_levothyroxine0_05Oral_Capsule',
 'prescribe_levothyroxine0_05Oral_Tablet',
 'prescribe_levothyroxine0_075Oral_Tablet',
 'prescribe_levothyroxine0_088Oral_Capsule',
 'prescribe_levothyroxine0_088Oral_Tablet',
 'prescribe_levothyroxine0_112Oral_Tablet',
 'prescribe_levothyroxine0_125Oral_Tablet',
 'prescribe_levothyroxine0_137Oral_Tablet',
 'prescribe_levothyroxine0_15Oral_Tablet',
 'prescribe_levothyroxine0_175Oral_Tablet',
 'prescribe_levothyroxine0_1Oral_Capsule',
 'prescribe_levothyroxine0_1Oral_Tablet',
 'prescribe_levothyroxine0_2Oral_Tablet',
 'prescribe_levothyroxine0_3Oral_Tablet',
 'prescribe_liothyronine0_005Oral_Tablet',
 'prescribe_liothyronine0_025Oral_Tablet',
 'prescribe_thyroid_USP_120Oral_Tablet',
 'prescribe_thyroid_USP_15Oral_Tablet',
 'prescribe_thyroid_USP_180Oral_Tablet',
 'prescribe_thyroid_USP_30Oral_Tablet',
 'prescribe_thyroid_USP_60Oral_Tablet',
 'prescribe_thy

In [10]:
sql_query_table_list = []
sql_query_list = []
table_count = 2
for name_strength_df in ingred_prescribe_category_list:
    sql_query_table_list.append('t'+str(table_count)+"."+name_strength_df)
    sql_query_list.append("LEFT JOIN (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS "+name_strength_df+
                          " FROM name_strength_df_distrib WHERE ingredient_prescribe_category LIKE'%"+name_strength_df+"%') AS t"+str(table_count)+
    " ON t1.age = t"+str(table_count)+".age AND t1.gender = t"+str(table_count)+".gender AND t1.state = t"+str(table_count)+".state")
    table_count += 1

sql_query_table_string = ', '.join(sql_query_table_list)
sql_query_string = ' '.join(sql_query_list)
sql_query_string

"LEFT JOIN (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS prescribe_levothyroxine0_013Oral_Capsule FROM name_strength_df_distrib WHERE ingredient_prescribe_category LIKE'%prescribe_levothyroxine0_013Oral_Capsule%') AS t2 ON t1.age = t2.age AND t1.gender = t2.gender AND t1.state = t2.state LEFT JOIN (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS prescribe_levothyroxine0_025Oral_Tablet FROM name_strength_df_distrib WHERE ingredient_prescribe_category LIKE'%prescribe_levothyroxine0_025Oral_Tablet%') AS t3 ON t1.age = t3.age AND t1.gender = t3.gender AND t1.state = t3.state LEFT JOIN (SELECT DISTINCT age, gender, state, percent_ingred_prescribe_patients AS prescribe_levothyroxine0_05Oral_Capsule FROM name_strength_df_distrib WHERE ingredient_prescribe_category LIKE'%prescribe_levothyroxine0_05Oral_Capsule%') AS t4 ON t1.age = t4.age AND t1.gender = t4.gender AND t1.state = t4.state LEFT JOIN (SELECT DISTINCT age, gender, state, percent_

In [11]:
drug_strength_df_distrib = db_query(f"""
        SELECT DISTINCT t1.age, t1.gender, t1.state, {sql_query_table_string}
        FROM name_strength_df_distrib AS t1
        {sql_query_string}
        """)
    
drug_strength_df_distrib.to_csv('thyroid_drug_strength_df_distrib.csv', index = False)
drug_strength_df_distrib

,age,gender,state,prescribe_levothyroxine0_013Oral_Capsule,prescribe_levothyroxine0_025Oral_Tablet,prescribe_levothyroxine0_05Oral_Capsule,prescribe_levothyroxine0_05Oral_Tablet,prescribe_levothyroxine0_075Oral_Tablet,prescribe_levothyroxine0_088Oral_Capsule,prescribe_levothyroxine0_088Oral_Tablet,...,prescribe_levothyroxine0_2Oral_Tablet,prescribe_levothyroxine0_3Oral_Tablet,prescribe_liothyronine0_005Oral_Tablet,prescribe_liothyronine0_025Oral_Tablet,prescribe_thyroid_USP_120Oral_Tablet,prescribe_thyroid_USP_15Oral_Tablet,prescribe_thyroid_USP_180Oral_Tablet,prescribe_thyroid_USP_30Oral_Tablet,prescribe_thyroid_USP_60Oral_Tablet,prescribe_thyroid_USP_90Oral_Tablet
0,12 - 18,Female,Alabama,NaN,42.846997,NaN,57.153003,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12 - 18,Female,Alaska,NaN,74.053157,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12 - 18,Female,Arizona,NaN,74.053157,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12 - 18,Female,Arkansas,NaN,42.846997,NaN,57.153003,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12 - 18,Female,California,NaN,74.053157,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,8 - 11,Male,Utah,NaN,NaN,NaN,NaN,NaN,NaN,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,8 - 11,Male,Virginia,NaN,68.340357,NaN,NaN,31.659643,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,8 - 11,Male,Washington,NaN,NaN,NaN,NaN,NaN,NaN,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,8 - 11,Male,West Virginia,NaN,68.340357,NaN,NaN,31.659643,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
